In [1]:
import pandas as pd
import numpy as np
import os
import glob
import joblib
from datetime import datetime, timedelta
from pathlib import Path

In [2]:
path = "/ml/algorithmlabs.csv"

In [3]:
file_list = glob.glob(f'{path}/*.csv')
name = pd.read_excel('ml/feature_korean_name.xlsx', sheet_name='promotion')

In [4]:
os.listdir(f'{path}')

['DAAT_DUTY_HISTORY.csv',
 'DAAT_ORDER.csv',
 'DAAT_PROJECT_HISTORY.csv',
 'DAAT_SUCCESSOR.csv',
 'DAAV_BASEINFO_DETAIL.csv',
 'DACT_USEDVACATION.csv',
 'DACT_WORK_RESULT_UPDATE.csv',
 'DAET_EDUCATION_REQUEST.csv',
 'DAJT_FINAL_RESULT.csv',
 'DAKT_MBO_USER.csv',
 'DANT_PROMOTION_USER.csv',
 'DHDT_MNTR_EXTRC_DTL.csv']

### DANT_PROMOTION_USER, DAAT_ORDER

In [6]:
print(file_list[10])
print(file_list[1])
user = pd.read_csv(file_list[7])
order = pd.read_csv(file_list[1])
user_data = user.copy()
order_data = order.copy()

c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DANT_PROMOTION_USER.csv
c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DAAT_ORDER.csv


In [331]:
year_list = []

for i in range(len(order_data)) : 
    year_list.append(int(str(order_data['DT_ORDER'][i])[0:4])-1)

order_data['DT_YEAR'] = year_list 

In [332]:
promotion = order_data[['ID_SABUN', 'DS_ORDER1', 'DT_YEAR']]

In [333]:
user_data.drop(user_data[user_data['DS_GROUP']=='3급 → 3-2급'].index, axis=0, inplace=True)

In [334]:
base = pd.merge(user_data, promotion, on=['ID_SABUN', 'DT_YEAR'], how='left')
base.drop(base[(base['CD_POSITION']==12) & (base['DS_ORDER1']!='승진')].index, axis=0, inplace=True)
base = base.reset_index(drop=True)

In [335]:
label_list = []
for i in range(len(base)) : 
    if base['DS_ORDER1'][i] != base['DS_ORDER1'][i] : 
        label_list.append(0)
    else :
        label_list.append(1)
        
base['LABEL'] = label_list
print(len(base))

221


In [336]:
base = base[['DT_YEAR', 'ID_SABUN', 'CD_BONBU', 'CD_DEPT', 'CD_JOBFAMILY', 'DT_GROUP_JOIN', 'DT_JOIN', 'DT_BEFORE_PROMOTION', 'DT_PROMOTION', 'NO_POSITION_YEAR',
             'NO_POINT_SUM', 'NO_POINT_LANGUAGE', 'NO_POINT_LICENSE', 'NO_POINT_REWARD', 'NO_POINT_DISCIPLINE', 'NO_POINT_EDU', 'NO_ADJUST_SUM',
             'NO_RANK', 'CD_ZONE', 'LABEL', 'DS_BONBU', 'DS_DEPT', 'DS_JOBFAMILY', 'DS_ZONE']]

base['ID_SABUN'] = base['ID_SABUN'].astype('str')

### DAAV_BASEINFO_DETAIL

In [7]:
print(file_list[4])
info = pd.read_csv(file_list[4])
info_data = info.copy()
info_data = info_data[['ID_SABUN', 'CD_JOBTYPE', 'CD_DUTY', 'DT_BIRTH', 'DS_BIRTHPLACE', 'CD_ADOPTYPE', 'YN_HANDICAP', 'YN_SUPPORTED', 'YN_FOREIGNER', 
                       'YN_GENDER', 'CD_JOBFAMILY_JOIN', 'DS_JOBTYPE', 'DS_DUTY', 'DS_ADOPTYPE', 'DS_JOBFAMILY_JOIN', 
                       'DS_HNAME', 'DS_POSITION', 'DS_OCCUPATION', 'DS_HANDPHONE', 'DS_EMAIL']]
info_data['ID_SABUN'] = info_data['ID_SABUN'].astype('str')

c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DAAV_BASEINFO_DETAIL.csv


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11084\295115312.py:2: DtypeWarning: Columns (36,38,39,53,167,171,173) have mixed types. Specify dtype option on import or set low_memory=False.
  info = pd.read_csv(file_list[4])


In [338]:
base = pd.merge(base, info_data, on=['ID_SABUN'], how='left')
print(len(base))

221


### DAAT_DUTY_HISTORY

In [8]:
print(file_list[0])
duty = pd.read_csv(file_list[0])
duty_data = duty.copy()

c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DAAT_DUTY_HISTORY.csv


In [340]:
duty_2020 = duty_data.loc[duty_data['DT_START'] < 20210101]
duty_2021 = duty_data.loc[duty_data['DT_START'] < 20220101]
duty_2022 = duty_data.loc[duty_data['DT_START'] < 20230101]

duty_list = []

for id in duty_2020['ID_SABUN'].unique().tolist() : 
    duty_list.append([id, 2020, duty_2020[duty_2020['ID_SABUN']==id]['CD_DUTY'].nunique()])
for id in duty_2021['ID_SABUN'].unique().tolist() : 
    duty_list.append([id, 2021, duty_2021[duty_2021['ID_SABUN']==id]['CD_DUTY'].nunique()])
for id in duty_2022['ID_SABUN'].unique().tolist() : 
    duty_list.append([id, 2022, duty_2022[duty_2022['ID_SABUN']==id]['CD_DUTY'].nunique()])

In [341]:
duty_temp = pd.DataFrame(duty_list, columns=['ID_SABUN', 'DT_YEAR', 'DUTY_CNT'])
duty_temp['ID_SABUN'] = duty_temp['ID_SABUN'].astype('str')
base = pd.merge(base, duty_temp, on = ['ID_SABUN', 'DT_YEAR'], how='left') 

### DAAT_PROJECT_HISTORY

In [9]:
print(file_list[2])
pro = pd.read_csv(file_list[2])
pro_data = pro.copy()

c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DAAT_PROJECT_HISTORY.csv


In [343]:
pro_data_2020 = pro_data.loc[pro_data['DT_START'] < 20210101]
pro_data_2021 = pro_data.loc[pro_data['DT_START'] < 20220101]
pro_data_2022 = pro_data.loc[pro_data['DT_START'] < 20230101]

In [344]:
base_2020 = base[base['DT_YEAR']==2020]
base_2021 = base[base['DT_YEAR']==2021]
base_2022 = base[base['DT_YEAR']==2022]

In [345]:
pro_cnt_2020 = pd.DataFrame(pro_data_2020['ID_SABUN'].value_counts().index.tolist(), columns=['ID_SABUN'])
pro_cnt_2020['PROJECT_CNT'] = pro_data_2020['ID_SABUN'].value_counts().values.tolist()
pro_cnt_2020['ID_SABUN'] = pro_cnt_2020['ID_SABUN'].astype('str')
base_2020 = pd.merge(base_2020, pro_cnt_2020, on=['ID_SABUN'], how='left')

In [346]:
pro_cnt_2021 = pd.DataFrame(pro_data_2021['ID_SABUN'].value_counts().index.tolist(), columns=['ID_SABUN'])
pro_cnt_2021['PROJECT_CNT'] = pro_data_2021['ID_SABUN'].value_counts().values.tolist()
pro_cnt_2021['ID_SABUN'] = pro_cnt_2021['ID_SABUN'].astype('str')
base_2021 = pd.merge(base_2021, pro_cnt_2021, on=['ID_SABUN'], how='left')

In [347]:
pro_cnt_2022 = pd.DataFrame(pro_data_2022['ID_SABUN'].value_counts().index.tolist(), columns=['ID_SABUN'])
pro_cnt_2022['PROJECT_CNT'] = pro_data_2022['ID_SABUN'].value_counts().values.tolist()
pro_cnt_2022['ID_SABUN'] = pro_cnt_2022['ID_SABUN'].astype('str')
base_2022 = pd.merge(base_2022, pro_cnt_2022, on=['ID_SABUN'], how='left')

In [348]:
base = pd.concat([base_2020, base_2021, base_2022], axis=0)
print(len(base))

221


### DAAT_SUCCESSOR

In [10]:
print(file_list[3])
sur = pd.read_csv(file_list[3])
sur_data = sur.copy()
sur_data = sur_data[['ID_SABUN', 'DT_YEAR']]
sur_data['SUCCESSOR_YN'] = 1
sur_data['ID_SABUN'] = sur_data['ID_SABUN'].astype('str')

c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DAAT_SUCCESSOR.csv


In [350]:
sur_data

,ID_SABUN,DT_YEAR,SUCCESSOR_YN
0,5029,2021,1
1,6467,2021,1
2,6415,2021,1
3,6376,2021,1
4,6414,2021,1
...,...,...,...
95,6505,2022,1
96,6460,2022,1
97,5599,2022,1
98,4905,2022,1


In [351]:
base = pd.merge(base, sur_data, on=['ID_SABUN', 'DT_YEAR'], how='left')
print(len(base))
base['SUCCESSOR_YN'] = base['SUCCESSOR_YN'].fillna(0)  

221


In [352]:
for i in sur_data['ID_SABUN'].unique().tolist() : 
    if i in base['ID_SABUN'].unique().tolist() : 
        print(i)

### DAJT_FINAL_RESULT & DAKT_MBO_USER

In [11]:
print(file_list[8])
wpe = pd.read_csv(file_list[8])
wpe_data = wpe.copy()

print(file_list[9])
mbo = pd.read_csv(file_list[9])
mbo_data = mbo.copy()

wpe_data = wpe_data[['ID_SABUN', 'DT_YEAR', 'NO_POINT_TOTAL_FIN', 'DS_POINT_TOTAL_FIN']]
mbo_data = mbo_data[['CD_USER', 'DT_YEAR', 'NO_FINAL_POINT', 'DS_FINAL_POINT']]
mbo_data.rename(columns = {'CD_USER' : 'ID_SABUN'}, inplace=True)

c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DAJT_FINAL_RESULT.csv
c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DAKT_MBO_USER.csv


In [354]:
point_data = pd.merge(wpe_data, mbo_data, on=['ID_SABUN', 'DT_YEAR'], how='outer')
point_data.dropna(subset = [ 'DS_POINT_TOTAL_FIN', 'NO_FINAL_POINT', 'DS_FINAL_POINT'], axis=0, how='all', inplace=True)
point_data['ID_SABUN'] = point_data['ID_SABUN'].astype('str')

In [355]:
three_years = []

for i in range(len(base)) : 
    temp_list = []
    temp_data = point_data[point_data['ID_SABUN']==base['ID_SABUN'][i]]
    main = base['DT_YEAR'][i]
    if main not in temp_data['DT_YEAR'].unique().tolist() : 
        temp_list.append(0)
        temp_list.append(0)
    else : 
        if ((temp_data[temp_data['DT_YEAR']==main]['NO_POINT_TOTAL_FIN'].values[0]==temp_data[temp_data['DT_YEAR']==main]['NO_POINT_TOTAL_FIN'].values[0]) |
        (temp_data[temp_data['DT_YEAR']==main]['DS_POINT_TOTAL_FIN'].values[0]==temp_data[temp_data['DT_YEAR']==main]['DS_POINT_TOTAL_FIN'].values[0])) : 
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['NO_POINT_TOTAL_FIN'].values[0])
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['DS_POINT_TOTAL_FIN'].values[0])
        else : 
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['NO_FINAL_POINT'].values[0])
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['DS_FINAL_POINT'].values[0])         
    
    main -= 1
    
    if main not in temp_data['DT_YEAR'].unique().tolist() : 
        temp_list.append(0)
        temp_list.append(0)
    else : 
        if ((temp_data[temp_data['DT_YEAR']==main]['NO_POINT_TOTAL_FIN'].values[0]==temp_data[temp_data['DT_YEAR']==main]['NO_POINT_TOTAL_FIN'].values[0]) |
        (temp_data[temp_data['DT_YEAR']==main]['DS_POINT_TOTAL_FIN'].values[0]==temp_data[temp_data['DT_YEAR']==main]['DS_POINT_TOTAL_FIN'].values[0])) : 
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['NO_POINT_TOTAL_FIN'].values[0])
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['DS_POINT_TOTAL_FIN'].values[0])
        else : 
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['NO_FINAL_POINT'].values[0])
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['DS_FINAL_POINT'].values[0])   
            
    main -= 1
    
    if main not in temp_data['DT_YEAR'].unique().tolist() : 
        temp_list.append(0)
        temp_list.append(0)
    else : 
        if ((temp_data[temp_data['DT_YEAR']==main]['NO_POINT_TOTAL_FIN'].values[0]==temp_data[temp_data['DT_YEAR']==main]['NO_POINT_TOTAL_FIN'].values[0]) |
        (temp_data[temp_data['DT_YEAR']==main]['DS_POINT_TOTAL_FIN'].values[0]==temp_data[temp_data['DT_YEAR']==main]['DS_POINT_TOTAL_FIN'].values[0])) : 
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['NO_POINT_TOTAL_FIN'].values[0])
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['DS_POINT_TOTAL_FIN'].values[0])
        else : 
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['NO_FINAL_POINT'].values[0])
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['DS_FINAL_POINT'].values[0])  
        
    three_years.append(temp_list)    

In [356]:
base[['FINAL_POINT_THIS_YEAR', 'FINAL_GRADE_THIS_YEAR', 'FINAL_POINT_ONE_YEAR', 'FINAL_GRADE_ONE_YEAR', 'FINAL_POINT_TWO_YEAR', 'FINAL_GRADE_TWO_YEAR']] = three_years

### DHDT_MNTR_EXTRC_DTL

In [12]:
print(file_list[11])
ext = pd.read_csv(file_list[11])
ext_data = ext.copy()
ext_data.dropna(subset=['TM_APPROVAL'], inplace=True)
ext_data['TM_APPROVAL'] = ext_data['TM_APPROVAL'].astype('int64')
ext_data = ext_data.reset_index(drop=True)

c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DHDT_MNTR_EXTRC_DTL.csv


In [358]:
ext_data = ext_data[(ext_data['DS_REM'].str.contains('식대')) | (ext_data['CD_REM']==2) | (ext_data['CD_REM']==14)]
ext_data = ext_data[ext_data['AM_TOT']>=50000]

In [359]:
month_cnt = []

for id in ext_data['ID_SABUN'].unique().tolist() : 
    temp_data = ext_data[ext_data['ID_SABUN']==id]
    avg = len(temp_data)/temp_data['YM_ACCOUNT'].nunique()
    month_cnt.append([id, avg])
    
ext_data_avg = pd.DataFrame(month_cnt, columns=['ID_SABUN', 'MNTR'])

In [360]:
ext_data_avg['ID_SABUN'] = ext_data_avg['ID_SABUN'].astype('int64')
ext_data_avg['ID_SABUN'] = ext_data_avg['ID_SABUN'].astype('str') 

In [361]:
base = pd.merge(base, ext_data_avg, on=['ID_SABUN'], how='left')

### missing values

In [362]:
# DT_PROMOTION 기준 DT_YEAR보다 뒤이면 GROUP_JOIN 사용, NULL값이면 DT_JOIN 사용
promotion_cnt = []

for i in range(len(base)) : 
    if base['DT_PROMOTION'][i] == base['DT_PROMOTION'][i] :
        if base['DT_YEAR'][i] <= int(str(base['DT_PROMOTION'][i])[0:4]) : 
            promotion_cnt.append(base['DT_YEAR'][i] - int(str(base['DT_GROUP_JOIN'][i])[0:4]))
        else : 
            promotion_cnt.append(base['DT_YEAR'][i] - int(str(base['DT_PROMOTION'][i])[0:4]))
    else : 
        promotion_cnt.append(base['DT_YEAR'][i] - int(str(base['DT_JOIN'][i])[0:4]))
        
base['BEFORE_PROMOTION_CNT'] = promotion_cnt

In [363]:
# 0411 수정
base.drop(['NO_RANK', 'FINAL_POINT_THIS_YEAR', 'FINAL_POINT_ONE_YEAR', 'FINAL_POINT_TWO_YEAR', 'DT_BEFORE_PROMOTION', 'DT_PROMOTION', 
           'NO_POINT_DISCIPLINE', 'NO_POINT_EDU', 'SUCCESSOR_YN'], axis=1, inplace=True)

In [364]:
# 0411 수정
missing_raw = base[base['DT_YEAR']!=2022]
missing_raw.drop(['ID_SABUN', 'DS_BONBU', 'DS_DEPT', 'DS_JOBFAMILY', 'DS_ZONE', 'DS_JOBTYPE', 'DS_DUTY', 'DS_ADOPTYPE', 'DS_JOBFAMILY_JOIN', 
                  'DS_HNAME', 'DS_POSITION', 'DS_OCCUPATION', 'DS_HANDPHONE', 'DS_EMAIL', 'DT_JOIN', 'DT_GROUP_JOIN', 'DT_YEAR'], axis=1, inplace=True)
missing_rate = (missing_raw.isnull().sum()/len(missing_raw)).to_dict()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8804\1311725033.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_raw.drop(['ID_SABUN', 'DS_BONBU', 'DS_DEPT', 'DS_JOBFAMILY', 'DS_ZONE', 'DS_JOBTYPE', 'DS_DUTY', 'DS_ADOPTYPE', 'DS_JOBFAMILY_JOIN',


In [365]:
base['CD_JOBFAMILY'] = base['CD_JOBFAMILY'].fillna(base['CD_JOBFAMILY'].mode()[0])
base['CD_JOBTYPE'] = base['CD_JOBTYPE'].fillna(base['CD_JOBTYPE'].mode()[0])
base['CD_DUTY'] = base['CD_DUTY'].fillna(base['CD_DUTY'].mode()[0])
base['DS_BIRTHPLACE'] = base['DS_BIRTHPLACE'].fillna(base['DS_BIRTHPLACE'].mode()[0])

In [366]:
base['YN_FOREIGNER'] = base['YN_FOREIGNER'].fillna('N')

In [367]:
base['DS_BIRTHPLACE'].replace({' ' : '서울', '대한민국' : '서울', '광주광역시' : '광주', '충청남도' : '충남', '강원도' : '강원', '경상남도' : '경남', '경기 광명' : '경기', 
                               '전북 남원' : '전북', '전북 익산' : '전북', ' 서울' : '서울'}, inplace=True)

In [368]:
# 0411 수정
zero_list = ['NO_POINT_LANGUAGE', 'NO_POINT_LICENSE', 'NO_POINT_REWARD', 'DUTY_CNT', 'PROJECT_CNT', 'MNTR']
base[zero_list] = base[zero_list].fillna(0)

In [369]:
def ch_grade(data, columns) : 
    data[columns] = data[columns].replace({'O' : 3, 'R' : 2, 'D' : 1, 0 : 2, 'A' : 3, 'B' : 2, 'C' : 1})
    data[columns] = data[columns].fillna(2)
    return data

In [370]:
ch_grade(base, 'FINAL_GRADE_THIS_YEAR')
ch_grade(base, 'FINAL_GRADE_ONE_YEAR')
ch_grade(base, 'FINAL_GRADE_TWO_YEAR')

,DT_YEAR,ID_SABUN,CD_BONBU,CD_DEPT,CD_JOBFAMILY,DT_GROUP_JOIN,DT_JOIN,NO_POSITION_YEAR,NO_POINT_SUM,NO_POINT_LANGUAGE,...,DS_OCCUPATION,DS_HANDPHONE,DS_EMAIL,DUTY_CNT,PROJECT_CNT,FINAL_GRADE_THIS_YEAR,FINAL_GRADE_ONE_YEAR,FINAL_GRADE_TWO_YEAR,MNTR,BEFORE_PROMOTION_CNT
0,2020,6455,305,50243812,1.0,20180813,20210111,6,0.000000,0.0,...,팀원,010-4557-0903,hjhan@hdc-dvp.com,1.0,0.0,2,2,2,4.333333,2
1,2020,6445,305,50301596,1.0,20191028,20191028,5,26.666667,0.0,...,팀원,010-5053-2853,6445@hdc-dvp.com,3.0,1.0,2,2,2,8.500000,1
2,2020,6444,305,50301596,1.0,20190909,20190909,7,26.666667,0.0,...,팀원,010-2689-3138,6444@hdc-dvp.com,1.0,0.0,2,2,2,6.333333,1
3,2020,6422,505,50278980,5.0,20190107,20190107,10,60.000000,0.0,...,팀원,010-7907-0736,gs-lim@hdc-dvp.com,3.0,5.0,3,2,2,4.500000,1
4,2020,6417,505,50298339,3.0,20181226,20181226,7,60.000000,0.0,...,사업소장,010-4233-9259,jinok@hdc-dvp.com,4.0,17.0,3,2,2,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,2022,6183,300,50279887,5.0,20111121,20111121,7,80.714286,0.0,...,팀원,010-7123-4442,namil@hdc-dvp.com,3.0,2.0,3,2,2,5.000000,6
217,2022,6318,605,50301688,6.0,19920113,20150801,26,77.857143,0.0,...,건축담당,010-2978-9000,artceo@hdc-dvp.com,0.0,9.0,1,1,2,0.000000,7
218,2022,6705,605,50301689,5.0,20221114,20221114,8,80.000000,0.0,...,안전공구장,01025213698,6705@hdc-dvp.com,1.0,0.0,2,2,2,0.000000,0
219,2022,6012,300,50298104,1.0,20090914,20090914,10,86.428571,0.0,...,팀원,010-3713-6562,jykim@hdc-dvp.com,5.0,8.0,3,3,2,11.000000,5


In [371]:
# 0411 수정
train_data = base[base['DT_YEAR']!=2022]
inference_data = base[base['DT_YEAR']==2022]
train_data.drop(['DS_HNAME', 'DS_POSITION', 'DS_OCCUPATION', 'DS_HANDPHONE', 'DS_EMAIL', 'DT_JOIN', 'DT_GROUP_JOIN', 'DT_YEAR'], axis=1, inplace=True)
inference_data.drop(['LABEL', 'DT_YEAR'], axis=1, inplace=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8804\616795564.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.drop(['DS_HNAME', 'DS_POSITION', 'DS_OCCUPATION', 'DS_HANDPHONE', 'DS_EMAIL', 'DT_JOIN', 'DT_GROUP_JOIN', 'DT_YEAR'], axis=1, inplace=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8804\616795564.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inference_data.drop(['LABEL', 'DT_YEAR'], axis=1, inplace=True)


In [372]:
name_dict = {}
for i in range(len(name)) : 
    name_dict[name['feature'][i]] = name['feature_name'][i]

In [373]:
today = datetime.today().strftime("%Y%m%d")
Path(f"/ml/algorithmlabs.inference/{today}/promotion/pickle").mkdir(parents=True, exist_ok=True)

train_data.to_csv(f"/ml/algorithmlabs.inference/{today}/promotion/train_data_promotion.csv", index=False)
inference_data.to_csv(f"/ml/algorithmlabs.inference/{today}/promotion/inference_data_promotion.csv", index=False)
joblib.dump(missing_rate, f"/ml/algorithmlabs.inference/{today}/promotion/pickle/missing_rate.pkl")
joblib.dump(name_dict, f"/ml/algorithmlabs.inference/{today}/promotion/pickle/name_dict.pkl")

['c:\\Users\\wongi\\Desktop\\알고리즘랩스\\프로젝트\\현대산업개발\\develop\\algorithmlabs.inference\\pickle_promotion\\name_dict.pkl']